# Add Snowpit Stratigraphy to Database
* then spatially enable inside the database

In [2]:
'''
  import external libraries
'''
import json
import sqlalchemy
import glob

import pandas as pd
import numpy as np
import glob
import os

In [3]:
'''
  define custom functions
'''
#Pull in and clean pit data (from excel -> clean tabular format)
def clean_snowex_pit_stratigraphy(pits):
    ''' 
      pits: list of pathnames to original snowex pit files (.xlsx)
    '''
    all_clean_dat=pd.DataFrame() #create empty data frame to fill in the loop
    for pit_pth in pits:
        dat=pd.read_excel(pit_pth,header=8) #read original table
        dat=dat.reset_index(drop=['index', 'Unnamed: 1']) #give index; reads in as NAN due to Excel formatting

        lst=list(range(9,17)) #list of column numbers desired
        lst.append(7)
        clean_dat=dat[lst].dropna(how='all') #drop nan rows created by "merged cells" data entry in excel

        #add date column
        pit_date=os.path.basename(pit_pth).split('_')[1]
        clean_dat['date']=pit_date

        #add UTM column
        utm_coord_df=pd.read_excel(pit_pth)
        utm_n=utm_coord_df['UTMN:'].values[0]
        utm_e=utm_coord_df['UTMN:'].values[2]#stored in same column
        clean_dat['utm_n']=utm_n
        clean_dat['utm_e']=utm_e

        #add pitname column
        pit_id=utm_coord_df['Location:'].values[4]
        clean_dat['pit_id']=pit_id

        #rename columns 
        clean_dat.columns=['snow_bottom_depth','min_grain_size','max_grain_size','mean_grain_size', 'grain_type', 'photo_exists','snow_wetness', 'stratig_comments', 'snow_top_depth', 'date', 'utm_n', 'utm_e', 'pit_id']#reorder to match order in entry sheet
        new_cols=clean_dat.columns.tolist()
        new_cols.remove('date') #move date to be first column
        new_cols.remove('snow_top_depth')
        new_cols=['date', 'snow_top_depth']+new_cols
        clean_dat=clean_dat[new_cols]
        all_clean_dat=all_clean_dat.append(clean_dat)
        
    return (all_clean_dat)

    
#Open database connection; return connection engine object
def open_database_connection(json_cred_pth):
    #json_cred_pth: path to json file holding database connection credentails
    with open(json_cred_pth) as f:
        db_conn_dict = json.load(f)

    cred_string = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**db_conn_dict)

    dbeng = sqlalchemy.create_engine(cred_string) #this is the connection "enginge" that we use to push and pull data
    return dbeng

In [19]:
#List all Excel Files containing desired snowpit data (profiles)
pits=glob.glob('Pit_Output/*/*/pit*.xlsx') #list files below
pits=[os.path.abspath(x) for x in pits] #store full path (not only relative)
#remove file throwing errors from list for now
pits.remove(r"C:\Users\emily\Documents\Python\Repos\snowex\Pit_Output\2017-02-15\PIT_39S\pit_20170215_39S.xlsx")

In [20]:
#Create database connection
json_cred_pth="SQL_access.json" #file with credentials
dbeng=open_database_connection(json_cred_pth) #this gets database connection engine

In [22]:
#Create "clean", single concatenated snowpit stratigraphy table
clean_strat=clean_snowex_pit_stratigraphy(pits)

In [25]:
#Add data to the database (may take a minute, if many rows)
clean_strat.to_sql('pit_stratigraphy', dbeng)

In [32]:
#Spatially-enable the table in the databse

#Add empty 'geom' column, of type "geometry", and in UTM 12N/ WGS 84 (same as incoming data)
dbeng.execute("""ALTER TABLE %s ADD COLUMN geom geometry(Point, 32612);""" %('pit_stratigraphy')) 
# populate the geometry field; reproject to lat/lon
dbeng.execute("""UPDATE %s SET geom = ST_Transform(ST_setSRID(ST_MakePoint(utm_e,utm_n),32612),4326);""" %('pit_stratigraphy'))